In [ ]:
import numpy as np
import pandas as pd
import os 
import matplotlib.pyplot as plt
from pandas import plotting
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cwd = os.getcwd()

In [ ]:
os.chdir('/content/drive/My Drive/Colab Notebooks/JanataHackCustomer')

In [ ]:
Dataset_train=pd.read_csv('Train_aBjfeNk.csv')
Dataset=pd.read_csv('Test_LqhgPWU.csv')

Dataset=pd.concat([Dataset_train, Dataset], axis=0, ignore_index=True, sort=False)

In [ ]:
Dataset.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [ ]:
Dataset.shape , Dataset_train.shape

((10695, 11), (8068, 11))

In [ ]:
Dataset.isnull().sum()

ID                    0
Gender                0
Ever_Married        190
Age                   0
Graduated           102
Profession          162
Work_Experience    1098
Spending_Score        0
Family_Size         448
Var_1               108
Segmentation       2627
dtype: int64

## AUTO EDA RESULTS

![alt text](https://drive.google.com/uc?export=view&id=11E-ze61OpfRvJhJvE5wIUaaDmR1KdoRL)

![alt text](https://drive.google.com/uc?export=view&id=18UTgg-Ed0bcSTljymkRa0iS9imaGehMM)

![alt text](https://drive.google.com/uc?export=view&id=11E-ze61OpfRvJhJvE5wIUaaDmR1KdoRL)

# MultiClass Approach

##Data Leak Detection, and list separation

In [ ]:
''' Finding any Data Leaks '''
len(Dataset['ID'].unique())

8363

In [ ]:
Dataset.shape[0]

10695

In [ ]:
'''These are the duplicate entires'''
Dataset.shape[0]-len(Dataset['ID'].unique())

2332

In [ ]:
''' Creating a new Dataframe with all duplicate ID's '''
Df = Dataset['ID'][Dataset['ID'].duplicated(keep=False)]
Df = Df.drop_duplicates(keep='last')

## NULL VALUE TREATEMENT

###IMPUTING

In [ ]:
Dataset['Ever_Married'].value_counts()
Dataset['Ever_Married'].fillna('NA', inplace=True)

In [ ]:
Dataset['Graduated'].value_counts()
Dataset['Graduated'].fillna('NA', inplace=True)

In [ ]:
Dataset['Profession'].value_counts()
Dataset['Profession'].fillna('NA', inplace=True)

In [ ]:
Dataset['Work_Experience'].fillna(-1.0, inplace=True)
Dataset['Work_Experience'].value_counts()

 1.0     3127
 0.0     3087
-1.0     1098
 9.0      613
 8.0      612
 2.0      373
 4.0      346
 3.0      337
 5.0      270
 6.0      265
 7.0      256
 14.0      66
 11.0      64
 10.0      64
 12.0      60
 13.0      57
Name: Work_Experience, dtype: int64

In [ ]:
Dataset['Family_Size'].fillna(-1, inplace=True)
Dataset['Family_Size'].value_counts()

 2.0    3158
 1.0    1965
 3.0    1952
 4.0    1823
 5.0     812
-1.0     448
 6.0     290
 7.0     122
 8.0      65
 9.0      60
Name: Family_Size, dtype: int64

In [ ]:
Dataset

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,-1.0,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,-1.0,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
10690,467954,Male,No,29,No,Healthcare,9.0,Low,4.0,Cat_6,NaN
10691,467958,Female,No,35,Yes,Doctor,1.0,Low,1.0,Cat_6,NaN
10692,467960,Female,No,53,Yes,Entertainment,-1.0,Low,2.0,Cat_6,NaN
10693,467961,Male,Yes,47,Yes,Executive,1.0,High,5.0,Cat_4,NaN


In [ ]:
Dataset['Var_1'].fillna('Unknown', inplace=True)
Dataset['Var_1'].value_counts()

Cat_6      6910
Cat_4      1475
Cat_3      1089
Cat_2       563
Cat_7       269
Cat_1       167
Cat_5       114
Unknown     108
Name: Var_1, dtype: int64

In [ ]:
# Dataset['Loan_Amount_uested']=Dataset['Loan_Amount_Requested'].str.replace("," ,'')

# Dataset['Loan_Amount_Requested']=Dataset['Loan_Amount_Requested'].astype(int)

# Dataset['Loan_Amount_Requested'].describe()

# Dataset['Loan_Amount_Requested']=np.log(Dataset['Loan_Amount_Requested'])

# Dataset['Months_Since_Deliquency'].describe()

# Dataset['Months_Since_Deliquency']=np.log(Dataset['Months_Since_Deliquency'], where = Dataset['Months_Since_Deliquency']!=0)

# Dataset['Annual_Income'].describe()

# Dataset['Annual_Income']=np.log(Dataset['Annual_Income'])

# Dataset['Number_Open_Accounts'].describe()

# Dataset['Number_Open_Accounts']=np.log(Dataset['Number_Open_Accounts'], where = Dataset['Number_Open_Accounts']!=0)


# Dataset['Total_Accounts']=np.log(Dataset['Total_Accounts'])

# Dataset['Home_Owner'].fillna('unknown', inplace=True)

# Dataset['Length_Employed']=Dataset['Length_Employed'].str.replace('years', '')

# Dataset['Length_Employed']=Dataset['Length_Employed'].replace('1 year', 1)

# Dataset['Length_Employed']=Dataset['Length_Employed'].replace('10+ ', 11)

# Dataset['Length_Employed']=Dataset['Length_Employed'].replace('< 1 year', 0)

# Dataset['Length_Employed'].fillna(21, inplace=True)

# Dataset['Length_Employed']=Dataset['Length_Employed'].astype(int)

# label_encode=LabelEncoder()

# Dataset['Gender']=label_encode.fit_transform(Dataset['Gender'])

# Dataset['Purpose_Of_Loan'].value_counts()

# Dataset['Purpose_Of_Loan']=Dataset['Purpose_Of_Loan'].map({'debt_consolidation':1, 'credit_card':2,
#                                                            'home_improvement':3, 'other':4,
#                                                            'major_purchase':5, 'small_business':6,
#                                                            'car':7, 'medical':8, 
#                                                            'moving':9, 'vacation':10,
#                                                            'wedding':11, 'house':12,
#                                                            'renewable_energy':13, 'educational':14})

In [ ]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10695 entries, 0 to 10694
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               10695 non-null  int64  
 1   Gender           10695 non-null  object 
 2   Ever_Married     10695 non-null  object 
 3   Age              10695 non-null  int64  
 4   Graduated        10695 non-null  object 
 5   Profession       10695 non-null  object 
 6   Work_Experience  10695 non-null  float64
 7   Spending_Score   10695 non-null  object 
 8   Family_Size      10695 non-null  float64
 9   Var_1            10695 non-null  object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 919.2+ KB


In [ ]:
Dataset=pd.get_dummies(Dataset, columns=['Gender', 'Ever_Married','Graduated','Profession','Var_1','Spending_Score'], drop_first=True)

In [ ]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10695 entries, 0 to 10694
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        10695 non-null  int64  
 1   Age                       10695 non-null  int64  
 2   Work_Experience           10695 non-null  float64
 3   Family_Size               10695 non-null  float64
 4   Segmentation              8068 non-null   object 
 5   Gender_Male               10695 non-null  uint8  
 6   Ever_Married_No           10695 non-null  uint8  
 7   Ever_Married_Yes          10695 non-null  uint8  
 8   Graduated_No              10695 non-null  uint8  
 9   Graduated_Yes             10695 non-null  uint8  
 10  Profession_Doctor         10695 non-null  uint8  
 11  Profession_Engineer       10695 non-null  uint8  
 12  Profession_Entertainment  10695 non-null  uint8  
 13  Profession_Executive      10695 non-null  uint8  
 14  Profes

In [ ]:
Dataset.isnull().sum()
Dataset.shape

(10695, 28)

In [ ]:
# Dataset['Account_ratio']=Dataset['Number_Open_Accounts']/Dataset['Total_Accounts']
# Dataset.head(5)

In [ ]:
Dataset_test=Dataset[Dataset['Segmentation'].isnull()==True]
Dataset=Dataset[Dataset['Segmentation'].isnull()!=True]

In [ ]:
Dataset_test.shape , Dataset.shape

((2627, 28), (8068, 28))

In [ ]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8068 entries, 0 to 8067
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        8068 non-null   int64  
 1   Age                       8068 non-null   int64  
 2   Work_Experience           8068 non-null   float64
 3   Family_Size               8068 non-null   float64
 4   Segmentation              8068 non-null   object 
 5   Gender_Male               8068 non-null   uint8  
 6   Ever_Married_No           8068 non-null   uint8  
 7   Ever_Married_Yes          8068 non-null   uint8  
 8   Graduated_No              8068 non-null   uint8  
 9   Graduated_Yes             8068 non-null   uint8  
 10  Profession_Doctor         8068 non-null   uint8  
 11  Profession_Engineer       8068 non-null   uint8  
 12  Profession_Entertainment  8068 non-null   uint8  
 13  Profession_Executive      8068 non-null   uint8  
 14  Professi

In [ ]:
#X=Dataset.drop(['Segmentation', 'ID'], axis=1)
#Y=Dataset['Segmentation']
X = Dataset.drop(['Segmentation'],axis=1)
Y = Dataset['Segmentation']

#X=X.iloc[:, :].values
#Y=Y.iloc[:].values

# imputer=SimpleImputer(missing_values=np.nan, strategy='mean')

# X[:, 2:3]=imputer.fit_transform(X[:, 2:3])

# X[:, 6:7]=imputer.fit_transform(X[:, 6:7])


In [ ]:
#rc=RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=10, n_jobs=-1, random_state=100)
#rc.fit(X_train,Y_train)
#Y_predict=rc.predict(X_test)

#print(f1_score(Y_test,Y_predict, average='weighted'))


In [ ]:
X=X.iloc[:, :].values
Y=Y.iloc[:].values

## INITIAL MODEL PROTOTYPING

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn import tree

In [ ]:
pd.DataFrame(X).isnull().sum()

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

#lg=LGBMClassifier(boosting_type='gbdt', max_depth=14, learning_rate=0.14, objective='multiclass', reg_alpha=0,
#                   reg_lambda=1, n_jobs=-1, random_state=100, n_estimators=600)

lg=LGBMClassifier(n_estimators=500)
#dt = tree.DecisionTreeClassifier()
lg.fit(X_train,Y_train)
Y_predict=lg.predict(X_test)

print(f1_score(Y_test,Y_predict, average='weighted'))
print(accuracy_score(Y_test,Y_predict))



/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5064976966447672
0.5086741016109045


In [ ]:
#X_main_test=Dataset_test.drop(['Segmentation', 'ID'], axis=1)
X_main_test=Dataset_test.drop(['Segmentation'], axis=1)
X_main_test=X_main_test.iloc[:, :].values

# X_main_test[:, 2:3]=imputer.fit_transform(X_main_test[:, 2:3])

# X_main_test[:, 6:7]=imputer.fit_transform(X_main_test[:, 6:7])

pd.DataFrame(X_main_test).isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
dtype: int64

In [ ]:
os.listdir()

['Test_LqhgPWU.csv',
 'Train_aBjfeNk.csv',
 'sample_submission.csv',
 'Report.html',
 'basic_solution_SCALING.csv',
 'basic_solution_impute_mean.csv',
 'basic_solution_ID.csv',
 'basic_solution_leaked.csv',
 'basic_solution_leaked_check.csv',
 'median_impute_check.csv',
 'nestimator300.csv',
 'nestimator500.csv',
 'nestimator500_ensemble.csv']

## Prediction

In [ ]:
Y_test_predict=lg.predict(X_main_test)

sample_submission=pd.read_csv('sample_submission.csv')
sample_submission['Segmentation']=Y_test_predict

#sample_submission.to_csv('basic_solution_leaked.csv', index=False)

## Editing the Predicted CSV


In [ ]:
leaked_prediction = sample_submission

In [ ]:
leaked_prediction

,ID,Segmentation
0,458989,1
1,458994,2
2,458996,0
3,459000,2
4,459001,3
...,...,...
2622,467954,3
2623,467958,1
2624,467960,0
2625,467961,1


In [ ]:
count = 0 
for i in Df:
  A = (Df.iloc[count] == leaked_prediction['ID'])
  leaked_prediction.Segmentation.iloc[A[::-1].idxmax()] = Dataset_train[Dataset_train.eq(i).any(1)].Segmentation.iloc[0]
  count+=1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
leaked_prediction

,ID,Segmentation
0,458989,B
1,458994,C
2,458996,A
3,459000,C
4,459001,C
...,...,...
2622,467954,D
2623,467958,A
2624,467960,0
2625,467961,B


In [ ]:
leaked_prediction.to_csv('nestimator800.csv', index=False)

## Cross Validation (Used, but doesnt effect the end result)


In [ ]:
X=pd.DataFrame(X)
Y=pd.DataFrame(Y)

In [ ]:
'''label encoding = y'''
label_encode=LabelEncoder()
Y= label_encode.fit_transform(Y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#label_encode.inverse_transform(Y)

In [ ]:
Dataset.shape

(8068, 28)

In [ ]:
model=LGBMClassifier(n_estimators=500)

In [ ]:
X=pd.DataFrame(X)
Y=pd.DataFrame(Y)
X_main_test=pd.DataFrame(X_main_test)

kf=KFold(n_splits=10, random_state=100, shuffle=True)

y_test_predict=0
f1_score1 = 0
acc_score1 = 0
j=1
result={}
for i, (train_index, test_index) in enumerate(kf.split(Dataset)):
    
   Y_train, Y_valid = Y.iloc[train_index], Y.iloc[test_index]
   X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
   
   print( "\nFold ", j)
    
   model=lg.fit(X_train,Y_train)
   pred = model.predict(X_valid)
   
   print(f1_score(Y_valid,pred, average='weighted'))
   print(accuracy_score(Y_valid,pred)) 
   f1_score1+=f1_score(Y_valid,pred, average='weighted')
   acc_score1+=accuracy_score(Y_valid,pred)
    
   y_test_predict+=model.predict(X_main_test)  
   result[j]=model.predict(X_main_test)
   j+=1
   
    
results=y_test_predict/10  
results=np.round(results, decimals=0)
print(f1_score1/10)

d = pd.DataFrame()
for i in range(1, 11):
    d = pd.concat([d,pd.DataFrame(result[i])],axis=1)
d.columns=['1','2','3','4','5','6','7','8','9','10']


Fold  1


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5192475949750658
0.5254027261462205

Fold  2
0.5014924239512717
0.5018587360594795

Fold  3
0.5297562632275656
0.5315985130111525

Fold  4
0.4971092007030907
0.4993804213135068

Fold  5
0.5272149743166348
0.530359355638166

Fold  6
0.5301313454201186
0.5278810408921933

Fold  7
0.525028358134088
0.5291201982651796

Fold  8
0.526514495746757
0.5291201982651796

Fold  9
0.5315328389289367
0.5347394540942928

Fold  10
0.5456221754461964
0.5483870967741935
0.5233649670849725


In [ ]:
re = d.mode(axis=1)[0]
re = pd.DataFrame(re).astype('int32')
re = label_encode.inverse_transform(re)
sample_submission=pd.read_csv('sample_submission.csv')
sample_submission['Segmentation']=re
#sample_submission.to_csv('C:/Users/UF145GV/Documents/DL/testing/Bank Statement/main_test.csv', index=False)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py
import plotly.graph_objs as go
from sklearn.cluster import KMeans
import warnings
import os

In [ ]:
X1=Dataset.drop(['Segmentation', 'ID'], axis=1).iloc[: , :].values
# Y=Dataset['Segmentation']
# X1 = df[['Age' , 'Spending Score (1-100)']].iloc[: , :].values
inertia = []
for n in range(1 , 11):
    algorithm = (KMeans(n_clusters = n ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
    algorithm.fit(X1)
    inertia.append(algorithm.inertia_)

In [ ]:
plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()

In [ ]:
algorithm = (KMeans(n_clusters = 4 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
algorithm.fit(X1)
labels1 = algorithm.labels_
centroids1 = algorithm.cluster_centers_

## Editing the Predicted CSV


In [ ]:
leaked_prediction = sample_submission

In [ ]:
leaked_prediction

,ID,Segmentation
0,458989,B
1,458994,C
2,458996,A
3,459000,C
4,459001,D
...,...,...
2622,467954,D
2623,467958,A
2624,467960,A
2625,467961,B


In [ ]:
count = 0 
for i in Df:
  A = (Df.iloc[count] == leaked_prediction['ID'])
  leaked_prediction.Segmentation.iloc[A[::-1].idxmax()] = Dataset_train[Dataset_train.eq(i).any(1)].Segmentation.iloc[0]
  count+=1

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
leaked_prediction

,ID,Segmentation
0,458989,B
1,458994,C
2,458996,A
3,459000,C
4,459001,C
...,...,...
2622,467954,D
2623,467958,A
2624,467960,A
2625,467961,B


In [ ]:
leaked_prediction.to_csv('nestimator500_ensemble.csv', index=False)